## 1. 테스트 데이터 전처리

In [1]:
import re
import json
import pandas as pd
from typing import List, Dict
from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams

INFO 04-22 06:25:26 [__init__.py:239] Automatically detected platform cuda.


In [2]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/ecommerce-function-calling-datasets-korean", split="train")

In [3]:
# 테스트 비율 설정
test_ratio = 0.2

# 전체 길이와 테스트 데이터 크기 계산
total_len = len(dataset)
test_size = int(total_len * test_ratio)

# 앞에서부터 테스트 데이터, 나머지는 학습 데이터
test_indices = list(range(test_size))
train_indices = list(range(test_size, total_len))

In [4]:
# OpenAI 포맷으로 변환 함수
def format_conversations(sample):
    return {
        "messages": [
            {"role": "system", "content": sample["system_prompt"]},
            *sample["messages"]
        ]
    }

# 분할 및 변환
train_dataset = [format_conversations(dataset[i]) for i in train_indices]
test_dataset = [format_conversations(dataset[i]) for i in test_indices]

# 리스트를 다시 HuggingFace Dataset 객체로 변환
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

# 결과 확인
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 311개, Test 77개


In [5]:
def to_chatml(data):
    """
    data: messages 리스트이거나 {"messages": [...]} 형태의 dict
    반환값: ChatML 포맷의 문자열
    """
    # data가 dict이고 'messages' 키가 있으면 messages 리스트를 꺼내고,
    # 아니면 data 자체를 messages 리스트로 간주
    messages = data.get("messages") if isinstance(data, dict) and "messages" in data else data

    parts = []
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        parts.append(f"<|im_start|>{role}\n{content}<|im_end|>")
    return "\n".join(parts)

In [6]:
def extract_examples(chatml: str) -> List[Dict[str, str]]:
    """
    ChatML 문자열에서 각 assistant 응답을 분리하여
    'input'과 'label' 쌍을 생성합니다.
    'input'은 해당 assistant 응답 직전까지의 모든 대화 + '<|im_start|>assistant',
    'label'은 해당 assistant의 응답 내용입니다.
    """
    examples: List[Dict[str, str]] = []
    pattern = re.compile(r'<\|im_start\|>assistant(.*?)(?=<\|im_end\|>)', re.DOTALL)

    for match in pattern.finditer(chatml):
        start_idx = match.start()
        input_text = chatml[:start_idx].strip() + '\n<|im_start|>assistant'
        label_text = match.group(1).strip()
        examples.append({
            "input": input_text,
            "label": label_text
        })

    return examples

In [7]:
prompt_lst = []
label_lst = []

for item in test_dataset:
    chatml = to_chatml(item)  # ChatML 문자열로 변환
    examples = extract_examples(chatml)  # assistant 응답 단위로 분리

    for ex in examples:
        prompt_lst.append(ex['input'])
        label_lst.append(ex['label'])

In [8]:
print(prompt_lst[10])

<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U006
오늘 날짜: 2024-02-02

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "add_to_cart", "description": "사용자의 장바구니에 지정된 상품(product_id)과 수량(quantity)을 추가합니다. 동일 상품이 이미 있으면 수량을 증가시키고, 새 항목으로 추가합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "장바구니에 상품을 추가할 사용자의 고유 식별자 (예: 'U001')"}, "product_id": {"type": "string", "description": "장바구니에 추가할 상품의 고유 식별자 (예: 'P003')"}, "quantity": {"type": "integer", "description": "추가할 상품 수량 (기본값: 1)", "default": 1, "minimum": 1}}, "required": ["user_id", "product_id"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "view_order_history", "description": "사용자의 전체 주문 내역을 반환합니다. 각 주문에 대해 주문 번호, 주문 일자, 총 결제 금액, 결제 상태, 배송 상태, 택배사, 운송장 번호, 배송 진행 단계, 주문에 포함된 상품명 목록을 제공

In [9]:
print(label_lst[10])

<tool_call>
{"name": "search_product", "arguments": {"keyword": "노트북"}}
</tool_call>


## 2. 모델 호출

In [10]:
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=2048,
    stop=["<|im_end|>"]
)

In [11]:
llm = LLM(model="Qwen/Qwen2.5-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

INFO 04-22 06:26:45 [config.py:585] This model supports multiple tasks: {'classify', 'generate', 'score', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 04-22 06:26:45 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

INFO 04-22 06:27:29 [weight_utils.py:281] Time spent downloading weights for Qwen/Qwen2.5-7B-Instruct: 38.528593 seconds


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-22 06:27:31 [loader.py:447] Loading weights took 1.84 seconds
INFO 04-22 06:27:31 [gpu_model_runner.py:1186] Model loading took 14.2487 GB and 40.861930 seconds
INFO 04-22 06:27:37 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/a207a5ab3c/rank_0_0 for vLLM's torch.compile
INFO 04-22 06:27:37 [backends.py:425] Dynamo bytecode transform time: 5.93 s
INFO 04-22 06:27:40 [backends.py:132] Cache the graph of shape None for later use
INFO 04-22 06:28:02 [backends.py:144] Compiling a graph for general shape takes 24.50 s
INFO 04-22 06:28:16 [monitor.py:33] torch.compile takes 30.43 s in total
INFO 04-22 06:28:16 [kv_cache_utils.py:566] GPU KV cache size: 953,616 tokens
INFO 04-22 06:28:16 [kv_cache_utils.py:569] Maximum concurrency for 32,768 tokens per request: 29.10x
INFO 04-22 06:28:42 [gpu_model_runner.py:1534] Graph capturing finished in 25 secs, took 0.48 GiB
INFO 04-22 06:28:42 [core.py:151] init engine (profile, create kv cache, warmup model) to

In [12]:
base_model_outputs = llm.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 655/655 [00:19<00:00, 32.84it/s, est. speed input: 65619.01 toks/s, output: 2315.51 toks/s] 


In [13]:
base_model_text_results = [sample.outputs[0].text.strip() for sample in base_model_outputs]

In [14]:
print(base_model_text_results[10])

<tool_call>
{"name": "search_product", "arguments": {"keyword": "노트북"}}
</tool_call>


## 3. 평가 결과 저장

In [16]:
df = pd.DataFrame({
    "prompt": prompt_lst,
    "label": label_lst,
    "output": base_model_text_results
})

# 저장
df.to_csv("base_model_evaluation_results.csv", index=False, encoding="utf-8-sig")

## 4. 평가

In [18]:
def evaluate_function_calls(labels, predictions):
    """
    펑션 콜링 성능을 평가하는 함수
    
    Parameters:
    -----------
    labels : list
        정답 레이블 목록
    predictions : list
        모델이 예측한 결과 목록
        
    Returns:
    --------
    dict
        tool_selection: 함수 이름 일치율
        params_selection: 파라미터 키 일치율 
        params_value_accuracy: 파라미터 값 일치율
        total_samples: 전체 tool_call 샘플 수
    """
    # 결과 저장할 딕셔너리 초기화
    results = {
        'tool_selection': {'correct': 0, 'total': 0},
        'params_selection': {'correct': 0, 'total': 0},
        'params_value_accuracy': {'correct': 0, 'total': 0}
    }
    
    # tool_call 형식만 필터링하기 위한 정규표현식
    tool_call_pattern = re.compile(r'<tool_call>(.*?)</tool_call>', re.DOTALL)
    
    # 전체 샘플 중 tool_call 샘플 수
    tool_call_count = 0
    
    for label, pred in zip(labels, predictions):
        # tool_call 형식인지 확인
        label_match = tool_call_pattern.search(label)
        pred_match = tool_call_pattern.search(pred)
        
        # 레이블이 tool_call이 아니면 건너뛰기
        if not label_match:
            continue
        
        tool_call_count += 1
        
        # 예측이 tool_call 형식이 아니면 모든 지표가 틀린 것으로 처리
        if not pred_match:
            results['tool_selection']['total'] += 1
            results['params_selection']['total'] += 1
            results['params_value_accuracy']['total'] += 1
            continue
        
        # JSON 파싱
        try:
            label_json = json.loads(label_match.group(1))
            pred_json = json.loads(pred_match.group(1))
        except json.JSONDecodeError:
            # JSON 파싱 오류 시 모든 지표가 틀린 것으로 처리
            results['tool_selection']['total'] += 1
            results['params_selection']['total'] += 1
            results['params_value_accuracy']['total'] += 1
            continue
        
        # 1. 함수 이름 일치 여부 (tool_selection)
        results['tool_selection']['total'] += 1
        if label_json.get('name') == pred_json.get('name'):
            results['tool_selection']['correct'] += 1
        
        # 2. 파라미터 키 일치 여부 (params_selection)
        # 개별 파라미터별로 맞고 틀림을 채점
        label_params = set(label_json.get('arguments', {}).keys())
        pred_params = set(pred_json.get('arguments', {}).keys())
        
        # 각 파라미터마다 평가를 위해 모든 파라미터 순회
        for param in label_params:
            results['params_selection']['total'] += 1
            if param in pred_params:
                results['params_selection']['correct'] += 1
        
        # 예측에만 있는 추가 파라미터도 틀린 것으로 평가
        for param in pred_params:
            if param not in label_params:
                results['params_selection']['total'] += 1
                # correct는 증가 안 함 (틀린 것이므로)
        
        # 3. 파라미터 값 일치 여부 (params_value_accuracy)
        # 존재하는 공통 파라미터에 대해서만 값 일치 여부 평가
        label_args = label_json.get('arguments', {})
        pred_args = pred_json.get('arguments', {})
        
        # 공통으로 존재하는 파라미터 키 찾기
        common_params = label_params.intersection(pred_params)
        
        if common_params:  # 공통 파라미터가 있는 경우에만 평가
            results['params_value_accuracy']['total'] += 1
            
            # 공통 파라미터의 값이 모두 일치하는지 확인
            values_match = True
            for key in common_params:
                if label_args.get(key) != pred_args.get(key):
                    values_match = False
                    break
            
            if values_match:
                results['params_value_accuracy']['correct'] += 1
    
    # 최종 결과 계산
    final_results = {}
    for metric, counts in results.items():
        if counts['total'] > 0:
            final_results[metric] = counts['correct'] / counts['total']
        else:
            final_results[metric] = 0.0
    
    final_results['total_samples'] = tool_call_count
    
    return final_results

In [19]:
labels = df['label'].to_list()
preds = df['output'].to_list()

results_with_errors = evaluate_function_calls(labels, preds)
for metric, value in results_with_errors.items():
    if metric != 'total_samples':
        print(f"{metric}: {value:.2%}")
    else:
        print(f"{metric}: {value}")

tool_selection: 78.06%
params_selection: 78.93%
params_value_accuracy: 77.84%
total_samples: 196
